In [1]:
import os
os.chdir('E:/BaiduSyncdisk/code/python/ppg/ppg2ecg_diff')
%load_ext autoreload

In [ ]:
from argparse import ArgumentParser
from torch.cuda import device_count
from torch.multiprocessing import spawn

from params import AttrDict, params as base_params
import numpy as np
import os
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from torch.utils.data import DataLoader

from config import cfg

import matplotlib.pyplot as plt

import util


#from denoising_model import ConditionalModel
import numpy as np
import os
import itertools
import argparse

from dataset import PPG2ECG_Dataset, PPG2ECG_Dataset_Eval
from util import MAE, NMAE, RMSE, NRMSE, PRD, DTW,PPC, cross_correlation,calculate_FD
import neurokit2 as nk
import torch_optimizer as optim 

In [3]:
#exec(open(os.path.join("scripts", "GetTrainTest.py"), encoding='utf-8').read())
exec(open(os.path.join("scripts", "common.py"), encoding='utf-8').read())
exec(open(os.path.join("scripts", "GetTrainTest_together.py"), encoding='utf-8').read())
#exec(open(os.path.join("scripts", "GetTrainTest.py"), encoding='utf-8').read())

In [4]:
c = cfg()

c.set_a_33 = False
c.set_a_select_channel = False


c.max_epoch =30

seed = 2023
##data

test_idx = 0
###
c.train = True
c.model_name ='unet'
c.rpeak_model_name = 'm_unet'
c.fs= 128
c.seg_len = int(128*4)
c.batch_size = 192*3
c.lr  =2e-4

c.loss_name = 'diff'#'L1'#'L1' #'mse'#'msc'
c.loss_weight = 0.99#0.99
model_save_dir = c.RESULT + '/model'


#c.remove_pl = False
c.train_all_channel = True
contain_label = True
c.mecg_label = False
remove_pl = False
c.show_pic = True
c.show_bar  = True

dataset_prefix = 'E:/code/ppg/resource/samples/'
model_dir ='E:/resourse/2024/ppg2ecg/model' #'E:/resource/2024/ppg/model'
#max_steps = 200*50#30*50 #30*50






c.min_max_norm = True
c.sample_k =1#10
#c.interp_method  ='scipy'
fast_sampling = False


c.r_con = True 
c.using_pred_r_con = True




datasets = ['01_BIDMC','02_CapnoBase']#[
c.db ='02_CapnoBase'#'03_DaLia' #'01_BIDMC'#'' Total_Partition

c.partition_path  ='D:/research/data/2024/PPG2ECG/partition/'+c.db+'_Partition_same_record.npy' ##v_Quality

rpeak_model_file ='m_unet_' +c.db + '_Rpeak'#
model_file = c.model_name + '_' + c.db + '_'+ str(int(c.r_con))  +'_together'
c.model_file = model_file 


In [5]:

# load partition
partition = np.load(c.partition_path, allow_pickle=True).item()

trainset = partition['trainset']
valset = partition['valset']

# get_dataloader function
def get_dataloader(batch_size, num_worker):
    trainloader_instance = PPG2ECG_Dataset(trainset, trainset, sampling_rate=c.fs, 
                                           min_max_norm=c.min_max_norm, z_score_norm=c.z_score_norm, interp=c.interp_method)
  #  trainloader_instance.__getitem__(0)
   # train_sampler = torch.utils.data.distributed.DistributedSampler(trainloader_instance, shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(trainloader_instance,
                                                   batch_size = c.batch_size,
                                                   shuffle = True,
                                                   num_workers = c.num_worker,
                                                   drop_last = False,
                                                   pin_memory = True)
    
    return train_dataloader


def get_test_dataloader():
    batch_size, num_worker = 100, 1
    test_instance = PPG2ECG_Dataset_Eval(valset, sampling_rate=c.fs, 
                                               min_max_norm=c.min_max_norm, z_score_norm=c.z_score_norm, interp=c.interp_method)
    test_dataloader = torch.utils.data.DataLoader(test_instance,
                                                       batch_size = batch_size,
                                                       shuffle = False,
                                                       num_workers = num_worker,
                                                       drop_last = False,
                                                       pin_memory = True)
    return test_dataloader



In [ ]:
util.speed_up()

pccs = []
msg = []
#setup(1,1)

print('database:',c.db)
# model_file =c.db + '_' + c.model_name  +'_'+str(test_idx)
max_pcc = 0
max_result = None
if c.train:
        # get generator
    train_dataloader = get_dataloader(c.batch_size, c.num_worker)
    
    print('training....')

    train(train_dataloader, base_params)
print('testing....')
test_dataloader = get_test_dataloader()
test(test_dataloader)
  